# Solution for Onedot Task
by Malik Sogukoglu
## Goal:
Bringing supplier data into the customer's data system format

#### Note!
Execute the cells in order from beginning to end.

## Make some imports

In [471]:
import pandas as pd
import numpy as np

## Read data from supplier (source data)
- Encoding is done with UTF-8

In [498]:
source = pd.read_json("supplier_car.json", lines=True, encoding="utf-8")

# 01. Preprocessing

In [500]:
source_preprocessed = source

### Supplier data: Column selection and roll-up of the embedded data fields
- The columns **MakeText**, **ModelText**, **ModelTypeText** contain information that can be integrated into the customer system - these are therefore retained.
- Furthermore, the source file is structured in such a way that for a given index, there are several entries with different values for **Attribute Names** and **Attribute Values**. These attributes (columns) have to be rolled up so that they appear as new columns.

In [501]:
source_preprocessed = source_preprocessed.pivot(index=['ID', 'MakeText','ModelText', 'ModelTypeText'], columns='Attribute Names', values='Attribute Values')

### Index is resetted

In [476]:
source_preprocessed = source_preprocessed.reset_index()

### Target dataset is loaded. This specifies the structure into which the source data is to be integrated.

In [477]:
target = pd.read_excel("Target Data.xlsx")

In [497]:
target

,carType,color,condition,currency,drive,city,country,make,manufacture_year,mileage,mileage_unit,model,model_variant,price_on_request,type,zip,manufacture_month,fuel_consumption_unit
0,Convertible / Roadster,White,Used,USD,LHD,Zuzwil,CH,Toyota,1983,0.0,kilometer,Celica,2800 i Supra,False,car,NaN,3.0,l_km_consumption
1,Convertible / Roadster,Other,Original Condition,USD,LHD,London,GB,Ferrari,2001,0.0,mile,550,NaN,True,car,NaN,NaN,NaN
2,Convertible / Roadster,White,Used,USD,LHD,Scotts Valley,US,Porsche,1989,0.0,mile,"911 ""G""",NaN,True,car,NaN,NaN,NaN
3,Convertible / Roadster,Blue,Used,USD,LHD,London,GB,Rolls-Royce,1961,0.0,kilometer,Silver Cloud II,NaN,False,car,NaN,NaN,NaN
4,Convertible / Roadster,Black,Original Condition,USD,LHD,Hong Kong,HK,Gumpert,2010,0.0,kilometer,Apollo,Apollo S 800hp,False,car,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7247,Targa,Red,Used,CHF,RHD,Tunbridge Wells,GB,Jaguar,1974,NaN,mile,XJC,Experimental Prototype,False,car,NaN,NaN,NaN
7248,Targa,Silver,Used,CHF,RHD,Singen,DE,Porsche,1997,NaN,kilometer,911 / 993 Turbo,993 Turbo,True,car,NaN,3.0,NaN
7249,Targa,Yellow,Used,CHF,RHD,BRUMMEN,NL,Ferrari,1980,NaN,kilometer,308 GTB,GTS,True,car,NaN,NaN,NaN
7250,Targa,Black,Restored,AUD,RHD,Huntington Station,US,Lotus,1977,NaN,NaN,Esprit,S1,False,car,NY,NaN,NaN


### Select columns in the source dataset that are also in the target dataset.

The columns  **currency**, **drive**, **country**, **ZIP** and **price_on_request** which are part of the target dataset, are not in the source dataset. Those columns are therefore no longer taken into account in this ETL process. 

However, for a given **city** (which is actually given in the source dataset), the corresponding **ZIP** could also be read out via an online service.

In [478]:
source_preprocessed.columns

Index(['ID', 'MakeText', 'ModelText', 'ModelTypeText', 'BodyColorText',
       'BodyTypeText', 'Ccm', 'City', 'Co2EmissionText', 'ConditionTypeText',
       'ConsumptionRatingText', 'ConsumptionTotalText', 'Doors',
       'DriveTypeText', 'FirstRegMonth', 'FirstRegYear', 'FuelTypeText', 'Hp',
       'InteriorColorText', 'Km', 'Properties', 'Seats',
       'TransmissionTypeText'],
      dtype='object', name='Attribute Names')

In [479]:
source_preprocessed = source_preprocessed[["BodyTypeText", "BodyColorText", "ConditionTypeText", "City", "MakeText", "FirstRegYear", "Km", "ModelText", "ModelTypeText", "FirstRegMonth", "ConsumptionTotalText"]] 

In [480]:
target.columns

Index(['carType', 'color', 'condition', 'currency', 'drive', 'city', 'country',
       'make', 'manufacture_year', 'mileage', 'mileage_unit', 'model',
       'model_variant', 'price_on_request', 'type', 'zip', 'manufacture_month',
       'fuel_consumption_unit'],
      dtype='object')

### Rename the selected columns in the source dataset according to the target dataset.

In [481]:
source_preprocessed = source_preprocessed.rename(columns={"BodyTypeText" : "carType", \
                                "BodyColorText" : "color",  \
                                "ConditionTypeText" : "condition",  \
                                "City" : "city",  \
                                "MakeText" : "make",  \
                                "FirstRegYear" : "manufacture_year",  \
                                "Km" : "mileage", \
                                "ModelText": "model", \
                                "ModelTypeText" : "model_variant", \
                                "BodyColorText" : "color" \
                               })

In [482]:
source_preprocessed

Attribute Names,carType,color,condition,city,make,manufacture_year,mileage,model,model_variant,FirstRegMonth,ConsumptionTotalText
0,Limousine,anthrazit,Occasion,Zuzwil,MERCEDES-BENZ,1999,31900,E 320,E 320 Elégance 4-Matic,1,11.5 l/100km
1,Kombi,anthrazit,Occasion,Zuzwil,AUDI,2008,25400,RS6,RS6 Avant 5.0 V10 quattro,7,14.0 l/100km
2,Kombi,anthrazit,Occasion,Zuzwil,AUDI,2002,38500,RS6,RS6 Avant quattro,10,14.6 l/100km
3,Coupé,anthrazit,Occasion,Zuzwil,CHEVROLET,2015,200,CORVETTE,Corvette Z06,6,12.7 l/100km
4,SUV / Geländewagen,anthrazit,Occasion,Zuzwil,PORSCHE,2010,2900,CAYENNE,Cayenne Turbo Techart Magnum Kit,1,11.5 l/100km
...,...,...,...,...,...,...,...,...,...,...,...
1148,Limousine,weiss mét.,Occasion,Zuzwil,MERCEDES-BENZ,2007,102000,S 65 AMG,S 65 AMG L,3,14.8 l/100km
1149,Limousine,weiss mét.,Occasion,Zuzwil,BMW,1998,39700,M5,M5,10,13.9 l/100km
1150,Limousine,weiss mét.,Occasion,Zuzwil,MERCEDES-BENZ,2005,99000,A 150,A 150 Classic,10,6.7 l/100km
1151,Coupé,weiss mét.,Occasion,Porrentruy,LAMBORGHINI,2008,5900,NaN,Reventon Coupé,4,null


# 02. Normalise

In [483]:
source_normalised = source_preprocessed.copy()

### Inspection of the variable categories of the source dataset

In [484]:
for col in source_normalised:
    print(source_normalised[col].unique())

['Limousine' 'Kombi' 'Coupé' 'SUV / Geländewagen' 'Cabriolet' 'Wohnkabine'
 'Kleinwagen' 'Kompaktvan / Minivan' 'Sattelschlepper' 'Pick-up' nan]
['anthrazit' 'anthrazit mét.' 'beige' 'beige mét.' 'blau' 'blau mét.'
 'bordeaux' 'bordeaux mét.' 'braun' 'braun mét.' 'gelb' 'gelb mét.' 'gold'
 'gold mét.' 'grau' 'grau mét.' 'grün' 'grün mét.' 'orange' 'orange mét.'
 'rot' 'rot mét.' 'schwarz' 'schwarz mét.' 'silber' 'silber mét.'
 'violett mét.' 'weiss' 'weiss mét.']
['Occasion' 'Oldtimer' 'Neu' 'Vorführmodell']
['Zuzwil' 'Sursee' 'Porrentruy' 'Safenwil' 'Basel' 'St. Gallen']
['MERCEDES-BENZ' 'AUDI' 'CHEVROLET' 'PORSCHE' 'FORD (USA)' 'ASTON MARTIN'
 'LOTUS' 'LAMBORGHINI' 'RUF' 'BMW' 'LAND ROVER' 'BMW-ALPINA' 'RENAULT'
 'BUGATTI' 'BENTLEY' 'GIOTTILINE' 'TRIUMPH' 'LINCOLN' 'FERRARI' 'HUMMER'
 'OPEL' 'JAGUAR' 'ALFA ROMEO' 'LANCIA' 'NSU' 'TOYOTA' 'DAIHATSU' 'CITROEN'
 'FORD' 'MASERATI' 'VW' 'MG' 'HONDA' 'SUBARU' 'VOLVO' 'MITSUBISHI' 'FIAT'
 'MORGAN' 'ROLLS-ROYCE' 'DAEWOO' 'MAZDA' 'CHRYSLER' 'D

### Inspection of the variable categories of the target dataset

In [485]:
for col in target:
    print(target[col].unique())

['Convertible / Roadster' 'Coupé' 'Custom' nan 'SUV' 'Other' 'Saloon'
 'Single seater' 'Station Wagon' 'Targa']
['White' 'Other' 'Blue' 'Black' 'Silver' 'Brown' 'Red' 'Gray' 'Green'
 'Beige' 'Yellow' 'Orange' 'Purple' 'Gold']
['Used' 'Original Condition' 'Restored' 'New' 'Used with guarantee'
 'Restoration Project']
['USD' 'JPY' 'GBP' 'EUR' 'CHF' 'AUD']
['LHD' 'RHD' nan]
['Zuzwil' 'London' 'Scotts Valley' 'Hong Kong' 'Bovenden' 'Isernhagen'
 'De Lier' 'Berlin' 'Waalwijk' 'Lyon' 'Aachen' 'Düsseldorf' 'Harelbeke'
 'Neustadt' 'Baierbrunn bei München' 'Bramley' 'CH-8852 Altendorf / SZ'
 'Singen' 'St. Louis' 'Erkelenz' 'HETEREN' 'Courbevoie' 'Mill' 'Overijse'
 'Paris' 'Huntington Station' 'Böblingen' 'Riedering' 'Preston'
 'Wuppertal' 'Langenfeld' 'Puget Sur Argens' 'Beamish' 'Hamburg'
 'Pleidelsheim' 'BRUMMEN' 'Aalst' 'Buckinghamshire UK' '8706 Meilen'
 'NUNSPEET' 'MOORDRECHT' 'Aalter' 'Köln' 'Gronsveld' 'CH-2900 Porrentury'
 '7302 Landquart' 'Olney' 'Turnhout' 'Maranello' nan
 'Dubai - Ve

### Normalisation of column "mileage_unit"

In the source data, the mileage was explicitly stated as kilometers for all rows under the column **Km**.  In this regard, the target dataset (in which the source data is to be transferred) has also the column **mileage_unit**. In order to transfer the source data into that target data-format, the value **kilometer** is declared explicitly as **mileage_unit** for all data entries in the source dataset.

In [486]:
source_normalised["mileage_unit"] = "kilometer"

...furthermore, the target data-format has the column **type**, whereby in the target dataset there are only entries of the type **cars**. By inspecting the categories of a similar column called **carType** in the source dataset, it appears that only cars are listed there as well --> therefore all entries in the source dataset are labeled as type **car**.

### Normalisation of column "type"

In [487]:
source_normalised["type"] = "car"

### Normalisation of column "fuel_consumption_unit"

#### Furthermore, according to the inspection from above, ...
the **car consumption** in the source dataset is given as **...l/km**. 
In order to convert these data into the target data-format, wherever the consumption was given in **l/km** (see column **ConsumptionTotalText** in the source dataset), the text **l_km_consumption** is inserted under the new column **fuel_consumption_unit**. If the consumption was not specified in the source dataset, the corresponding text field is left empty.

In [488]:
source_normalised['fuel_consumption_unit'] = np.where(source_normalised['ConsumptionTotalText'].str.contains("l/100km"), "l_km_consumption", "")

#### Remove column ConsumptionTotalText as it has been replaced by "fuel_consuption_unit" in the target data format.

In [489]:
source_normalised = source_normalised.drop(['ConsumptionTotalText'], axis=1)

### Normalisation of column "color"

When normalising the **color** column, it is noticeable that the target dataset contains significantly fewer colors than the source dataset. 
The car colours are now named according to the colors in the target set, whereby distinctions such as **metallic/non-metallic are no longer made**, i.e. the granulation is no longer as deep. In addition, the color names are **translated from German into English**.

- Source **color**-Levels: 
    - ['anthrazit' 'anthrazit mét.' 'beige' 'beige mét.' 'blau' 'blau mét.'
 'bordeaux' 'bordeaux mét.' 'braun' 'braun mét.' 'gelb' 'gelb mét.' 'gold'
 'gold mét.' 'grau' 'grau mét.' 'grün' 'grün mét.' 'orange' 'orange mét.'
 'rot' 'rot mét.' 'schwarz' 'schwarz mét.' 'silber' 'silber mét.'
 'violett mét.' 'weiss' 'weiss mét.']

----[:normalise]---->
    
- Target **color**-Levels: 
    - ['White' 'Other' 'Blue' 'Black' 'Silver' 'Brown' 'Red' 'Gray' 'Green'
 'Beige' 'Yellow' 'Orange' 'Purple' 'Gold']

In [490]:
source_normalised['color'] = source_normalised['color'].replace({'anthrazit':'Gray', \
                                  'anthrazit mét.':'Gray', \
                                  'beige' : 'Beige' , \
                                  'beige mét.' : 'Beige', \
                                  'blau' : 'Blue' , \
                                  'blau mét.' : 'Blue' , \
                                  'bordeaux' : 'Purple', \
                                  'bordeaux mét.' : 'Purple', \
                                  'braun' : 'Brown', \
                                  'braun mét.' : 'Brown', \
                                  'gelb' : 'Yellow', \
                                  'gelb mét.' : 'Yellow', \
                                  'gold' : 'Gold', \
                                  'gold mét.': 'Gold' , \
                                  'grau': 'Gray',\
                                  'grau mét.' : 'Gray', \
                                  'grün' : 'Green' , \
                                  'grün mét.' : 'Green' , \
                                  'orange' : 'Orange' , \
                                  'orange mét.' : 'Orange' , \
                                  'rot' : 'Red' , \
                                  'rot mét.' : 'Red' , \
                                  'schwarz' : 'Black' , \
                                  'schwarz mét.' : 'Black' , \
                                  'silber' : 'Silver' , \
                                  'silber mét.' : 'Silver' , \
                                  'violett mét.' : 'Purple' , \
                                  'weiss' : 'White' , \
                                  'weiss mét.' : 'White'

                                 }
                                )

### TODO: normalisation of column "Condition"
For the conversion of column **condition**, a little more background knowledge is required. For example, it is not clear whether an **occasion** in the source dataset means a **used**, or rather a **uses with guarantee** in the target dataset.


- Source **condition**-Levels: 
    - ['Occasion' 'Oldtimer' 'Neu' 'Vorführmodell']

----[:normalise]---->

- Target **condition**-Levels: 
    - ['Used' 'Original Condition' 'Restored' 'New' 'Used with guarantee'
 'Restoration Project']

In [503]:
### Example code
# source_normalised = source_normalised['condition'].replace({'Occasion':'Used', 'Neu':'New'})

### TODO: normalisation of column "carType" 
Also for **carType** it is not quite clear which category in the source dataset corresponds to those in the target dataset - in this context, it might be possible to contact the customer or the supplier, to gain a deeper understanding of this topic.

- Source **carType**-Levels: 
    - ['Limousine' 'Kombi' 'Coupé' 'SUV / Geländewagen' 'Cabriolet' 'Wohnkabine'
 'Kleinwagen' 'Kompaktvan / Minivan' 'Sattelschlepper' 'Pick-up' nan]

----[:normalise]---->

- Target **carType**-Levels: 
    - ['Convertible / Roadster' 'Coupé' 'Custom' nan 'SUV' 'Other' 'Saloon'
 'Single seater' 'Station Wagon' 'Targa']

In [492]:
source_normalised

Attribute Names,carType,color,condition,city,make,manufacture_year,mileage,model,model_variant,FirstRegMonth,mileage_unit,type,fuel_consumption_unit
0,Limousine,Gray,Occasion,Zuzwil,MERCEDES-BENZ,1999,31900,E 320,E 320 Elégance 4-Matic,1,kilometer,car,l_km_consumption
1,Kombi,Gray,Occasion,Zuzwil,AUDI,2008,25400,RS6,RS6 Avant 5.0 V10 quattro,7,kilometer,car,l_km_consumption
2,Kombi,Gray,Occasion,Zuzwil,AUDI,2002,38500,RS6,RS6 Avant quattro,10,kilometer,car,l_km_consumption
3,Coupé,Gray,Occasion,Zuzwil,CHEVROLET,2015,200,CORVETTE,Corvette Z06,6,kilometer,car,l_km_consumption
4,SUV / Geländewagen,Gray,Occasion,Zuzwil,PORSCHE,2010,2900,CAYENNE,Cayenne Turbo Techart Magnum Kit,1,kilometer,car,l_km_consumption
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,Limousine,White,Occasion,Zuzwil,MERCEDES-BENZ,2007,102000,S 65 AMG,S 65 AMG L,3,kilometer,car,l_km_consumption
1149,Limousine,White,Occasion,Zuzwil,BMW,1998,39700,M5,M5,10,kilometer,car,l_km_consumption
1150,Limousine,White,Occasion,Zuzwil,MERCEDES-BENZ,2005,99000,A 150,A 150 Classic,10,kilometer,car,l_km_consumption
1151,Coupé,White,Occasion,Porrentruy,LAMBORGHINI,2008,5900,NaN,Reventon Coupé,4,kilometer,car,


# 3. Integration
Up to this step, the source dataset was prepared for integration into the target data-format. The following should be noted about the **adjusted source dataset**:
- Only columns of source dataset that fit into target dataset are considered.
- No additional data was added or data was removed.
- Source data is preprocessed and normalised.

In [493]:
integrated = pd.concat([source_normalised, target])

In [496]:
integrated

,carType,color,condition,city,make,manufacture_year,mileage,model,model_variant,FirstRegMonth,mileage_unit,type,fuel_consumption_unit,currency,drive,country,price_on_request,zip,manufacture_month
0,Limousine,Gray,Occasion,Zuzwil,MERCEDES-BENZ,1999,31900,E 320,E 320 Elégance 4-Matic,1,kilometer,car,l_km_consumption,NaN,NaN,NaN,NaN,NaN,NaN
1,Kombi,Gray,Occasion,Zuzwil,AUDI,2008,25400,RS6,RS6 Avant 5.0 V10 quattro,7,kilometer,car,l_km_consumption,NaN,NaN,NaN,NaN,NaN,NaN
2,Kombi,Gray,Occasion,Zuzwil,AUDI,2002,38500,RS6,RS6 Avant quattro,10,kilometer,car,l_km_consumption,NaN,NaN,NaN,NaN,NaN,NaN
3,Coupé,Gray,Occasion,Zuzwil,CHEVROLET,2015,200,CORVETTE,Corvette Z06,6,kilometer,car,l_km_consumption,NaN,NaN,NaN,NaN,NaN,NaN
4,SUV / Geländewagen,Gray,Occasion,Zuzwil,PORSCHE,2010,2900,CAYENNE,Cayenne Turbo Techart Magnum Kit,1,kilometer,car,l_km_consumption,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7247,Targa,Red,Used,Tunbridge Wells,Jaguar,1974,NaN,XJC,Experimental Prototype,NaN,mile,car,NaN,CHF,RHD,GB,False,NaN,NaN
7248,Targa,Silver,Used,Singen,Porsche,1997,NaN,911 / 993 Turbo,993 Turbo,NaN,kilometer,car,NaN,CHF,RHD,DE,True,NaN,3.0
7249,Targa,Yellow,Used,BRUMMEN,Ferrari,1980,NaN,308 GTB,GTS,NaN,kilometer,car,NaN,CHF,RHD,NL,True,NaN,NaN
7250,Targa,Black,Restored,Huntington Station,Lotus,1977,NaN,Esprit,S1,NaN,NaN,car,NaN,AUD,RHD,US,False,NY,NaN


### Write preprocessed, normalised and integrated data into Excel file

In [495]:
path = r"onedot_output_malik.xlsx"
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
source_preprocessed.to_excel(writer, sheet_name = "01_Preprocessed")
source_normalised.to_excel(writer, sheet_name = "02_Normalised")
integrated.to_excel(writer, sheet_name = "03_Integrated")
writer.save()
writer.close()

C:\Users\malik\anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
